Created by: [SmirkyGraphs](https://smirkygraphs.github.io/). Code: [Github](https://github.com/SmirkyGraphs/Python-Notebooks). Source: [FBI NICS](https://www.fbi.gov/file-repository/nics_firearm_checks_-_month_year_by_state_type.pdf/view) | [BuzzfeedNews](https://github.com/BuzzFeedNews/nics-firearm-background-checks/blob/master/data/nics-firearm-background-checks.csv).
<hr>

# Rhode Island NICS Background Checks

This Notebook contains code releated to the data cleaning for my [Tableau dashboard](https://public.tableau.com/profile/smirkygraphs#!/vizhome/RhodeIslandNICSBackgroundChecks/main). The code automatically downloads the most recent data from BuzzfeedNew's Github which parses the monthly PDF's from the FBI NICS. The data is shaped for use in Tableau (tall instead of wide) and cleaned up a bit. Additionally at the bottom their is code that will rank all states by their YoY increase and YTD increase. An issue with this is that every state has different laws so comparing is difficult, for example Kentucky does a recheck on every permit every month.

<hr>

In [1]:
import pandas as pd
from pathlib import Path

In [2]:
# make file paths
Path("./data/raw").mkdir(parents=True, exist_ok=True)
Path("./data/clean").mkdir(parents=True, exist_ok=True)

# download data from github
file = 'nics-firearm-background-checks/master/data/nics-firearm-background-checks.csv'
df = pd.read_csv(f'https://raw.githubusercontent.com/BuzzFeedNews/{file}')
df.to_csv('./data/raw/nics-firearm-background-checks.csv', index=False)

In [3]:
# load data and filter for rhode island
df = pd.read_csv('./data/raw/nics-firearm-background-checks.csv', parse_dates=['month'])

# melt (pivot) data turning it from wide to long for tableau & remove total
df = df.drop(columns=['totals']).rename(columns={'month':'date'})
df = df.melt(id_vars=['state', 'date'])

# add sales type
types = ['prepawn', 'redemption', 'returned', 'rentals', 'private_sale', 'return_to_seller']
for sale_type in types:
    df.loc[df['variable'].str.startswith(sale_type), 'type'] = sale_type
df['type'] = df['type'].fillna('FFL/LEA')

# remove 1998 as it only has nov/dec
df = df[df['date'].dt.year > 1998]

# add weapon style
df.loc[df['variable'].str.startswith('permit'), 'style'] = 'permit'
df.loc[df['variable'].str.endswith('handgun'), 'style'] = 'handgun'
df.loc[df['variable'].str.endswith('long_gun'), 'style'] = 'long_gun'
df.loc[df['variable'].str.endswith('other'), 'style'] = 'other'
df.loc[df['variable'] == 'multiple', 'style'] = 'multiple'
df.loc[df['variable'] == 'admin', 'style'] = 'admin'

# save rhode island extract
ri = df[df['state']=='Rhode Island']
ri.to_csv('./data/clean/ri-nics-firearm-clean.csv', index=False)

In [4]:
# drop unwanted states & keep states + dc
drop = ['Guam', 'Mariana Islands', 'Puerto Rico', 'Virgin Islands']
states = [x for x in df['state'].unique().tolist() if x not in drop]
df = df[df['state'].isin(states)]

# save all states data
df.to_csv('./data/clean/nics-firearm-clean.csv', index=False)

# get percent change from prior year (full and ytd)
def full_ranks(df):
    df = df.groupby([df['state'], df['date'].dt.year]).sum()
    df['pct'] = df.pct_change()
    
    df = df.rename(columns={'value': 'fy_value', 'pct': 'fy_pct'})
    
    return df

def ytd_ranks(df):
    # filter all years to current month
    df = df[df['date'].dt.month <= df['date'].max().month]
    df = df.groupby([df['state'], df['date'].dt.year]).sum()
    df['pct'] = df.pct_change()
    
    df = df.rename(columns={'value': 'ytd_value', 'pct': 'ytd_pct'})
    
    return df

# combine and save
fy_df = full_ranks(df)
ytd_df = ytd_ranks(df)
df = pd.concat([fy_df, ytd_df], axis=1).reset_index()
df.to_csv('./data/clean/state-ranks.csv', index=False)

In [6]:
df[df['state']=='Rhode Island']

,state,date,fy_value,fy_pct,ytd_value,ytd_pct
936,Rhode Island,1999,10677.0,-0.982766,4832.0,-0.992201
937,Rhode Island,2000,9414.0,-0.118292,4701.0,-0.027111
938,Rhode Island,2001,10233.0,0.086998,4220.0,-0.102319
939,Rhode Island,2002,9521.0,-0.069579,4616.0,0.093839
940,Rhode Island,2003,9660.0,0.014599,4661.0,0.009749
941,Rhode Island,2004,10054.0,0.040787,4894.0,0.049989
942,Rhode Island,2005,9649.0,-0.040282,4714.0,-0.036780
943,Rhode Island,2006,9936.0,0.029744,4918.0,0.043275
944,Rhode Island,2007,10222.0,0.028784,5218.0,0.061000
945,Rhode Island,2008,12405.0,0.213559,5636.0,0.080107


In [14]:
ri['variable'].unique()

array(['permit', 'permit_recheck', 'handgun', 'long_gun', 'other',
       'multiple', 'admin', 'prepawn_handgun', 'prepawn_long_gun',
       'prepawn_other', 'redemption_handgun', 'redemption_long_gun',
       'redemption_other', 'returned_handgun', 'returned_long_gun',
       'returned_other', 'rentals_handgun', 'rentals_long_gun',
       'private_sale_handgun', 'private_sale_long_gun',
       'private_sale_other', 'return_to_seller_handgun',
       'return_to_seller_long_gun', 'return_to_seller_other'],
      dtype=object)